# Preparação de Dados (N1)

## 🗂️ [ Projeto ]

**Nome do Projeto:** [nome do projeto]  
**Dataset:** [Nome do dataset ou fonte de dados]  
**Objetivo Geral:** [Breve Explicação do propósito — ex.: “Analisar o comportamento de churn e identificar fatores de retenção de clientes.”]  

---

## 📋 Descrição Geral do Tratamento

Nesta etapa serão realizados os principais **processos de limpeza, transformação e padronização dos dados**.  
O foco é garantir que o dataset esteja **consistente, estruturado e pronto para análises posteriores**.

**Tarefas previstas:**  
- Leitura e diagnóstico inicial do dataset.  
- Tratamento de valores nulos, duplicados e outliers.  
- Conversão de tipos e padronização de nomes de colunas.  
- Codificação de variáveis categóricas e normalização de numéricas (quando necessário).  
- Geração de artefatos intermediários e relatório de qualidade dos dados.

## 🔧 Configuração do Projeto (Bootstrap)

Esta etapa realiza o **bootstrap do ambiente de execução**, preparando o notebook para funcionar em qualquer máquina — garantindo **reprodutibilidade**, **organização** e **autonomia**.  
Todas as dependências, caminhos e configurações são ajustadas automaticamente, sem necessidade de modificações manuais.

---

### ⚙️ Etapas realizadas

#### 🧭 1. Localização da raiz do projeto  
- Busca o arquivo `config/defaults.json` subindo diretórios até encontrá-lo.  
- Define `PROJECT_ROOT` como ponto de referência global para todos os caminhos do projeto.

#### 🧩 2. Validação da estrutura `utils/`  
- Verifica se existem a pasta `utils/` e o arquivo `__init__.py` (criado automaticamente, se ausente).  
- Garante que o pacote seja reconhecido pelo Python, permitindo importar `utils.utils_data` sem erros.

#### 🧱 3. Registro no `sys.path`  
- Adiciona o diretório raiz (`PROJECT_ROOT`) ao `sys.path`, habilitando o uso de módulos internos em qualquer ambiente.  
- Exibe o caminho detectado e o status do registro.

#### ♻️ 4. Importação e recarregamento das utilidades  
- Importa o módulo `utils.utils_data` e o recarrega via `importlib.reload`.  
- Essa abordagem assegura que o notebook use **sempre a versão mais recente** do arquivo durante o desenvolvimento iterativo.

#### 🧾 5. Inicialização do sistema de logs  
- Cria um log unificado em `reports/data_preparation.log`.  
- Cada etapa importante (carregamento, limpeza, exportação etc.) é registrada tanto no **console** quanto em **arquivo**, facilitando a rastreabilidade do processo.

#### ⚙️ 6. Carregamento das configurações globais  
- Lê os parâmetros de `config/defaults.json` e, se existir, aplica substituições de `config/local.json`.  
- Esses parâmetros controlam o comportamento de cada etapa (tratamento de nulos, tipagem, outliers, encoding etc.).

#### 📂 7. Resolução de diretórios e arquivos padrão  
- Garante a existência das pastas principais:  
  `data/raw`, `data/interim`, `data/processed`, `reports`, `artifacts`, `prints`, `dashboards`.  
- Define os arquivos de saída padrão (interim e processed), mantendo compatibilidade entre notebooks.

#### 🔄 8. Ambiente reprodutível e consistente  
- Define a semente aleatória (`RANDOM_SEED = 42`) para resultados reprodutíveis.  
- Ajusta as opções de exibição do pandas (`max_columns` e `display.width`) para uma visualização mais confortável.

---

### ✅ Resultado esperado

Ao final desta célula:

- O projeto é **reconhecido automaticamente**, independentemente do ambiente de execução.  
- O módulo `utils_data.py` é importado com todas as funções utilitárias disponíveis.  
- As pastas e arquivos padrão são criados e configurados.  
- O log central começa a registrar todas as ações executadas nas próximas etapas.  

---

> 💡 **Resumo:** Este design modular permite que o mesmo notebook seja executado em qualquer repositório que siga a estrutura do template, sem ajustes manuais de caminho ou imports.  


In [1]:
# -*- coding: utf-8 -*-
# Bootstrap do projeto — encontra a raiz, injeta no sys.path e garante utils/__init__.py

from pathlib import Path
import json
import sys

import pandas as pd


def _find_up(relative_path: str, start: Path | None = None) -> Path | None:
    start = start or Path.cwd()
    rel = Path(relative_path)
    for base in (start, *start.parents):
        cand = base / rel
        if cand.exists():
            return cand
    return None

# 1) Descobrir a raiz a partir do config/defaults.json
_cfg = _find_up("config/defaults.json")
if _cfg is None:
    raise FileNotFoundError(
        "config/defaults.json não encontrado. Confirme a estrutura do projeto."
    )
PROJECT_ROOT = _cfg.parent.parent.resolve()
print(f"[INFO] PROJECT_ROOT: {PROJECT_ROOT}")

# 2) Garantir pasta utils/ e __init__.py
UTILS_DIR = PROJECT_ROOT / "utils"
INIT_FILE = UTILS_DIR / "__init__.py"
if not UTILS_DIR.exists():
    raise ModuleNotFoundError(f"Pasta não encontrada: {UTILS_DIR} (crie 'utils' na raiz).")
if not INIT_FILE.exists():
    # cria um __init__.py vazio se não existir (facilita em notebooks)
    INIT_FILE.write_text("", encoding="utf-8")
    print(f"[INFO] Criado: {INIT_FILE}")

# 3) Injetar a raiz no sys.path
root_str = str(PROJECT_ROOT)
if root_str not in sys.path:
    sys.path.insert(0, root_str)
print(f"[INFO] sys.path ok. utils: {UTILS_DIR}")

# 4) Importar utils.utils_data
import importlib, utils.utils_data as ud
importlib.reload(ud)  # garante versão mais recente ao iterar no notebook

# 5) Configurar logging base do notebook
import logging
from datetime import datetime
LOG_FILE = (PROJECT_ROOT / "reports" / "data_preparation.log")
LOG_FILE.parent.mkdir(parents=True, exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[logging.StreamHandler(sys.stdout), logging.FileHandler(LOG_FILE, encoding="utf-8")],
    force=True,
)
logger = logging.getLogger(__name__)
logger.info("Bootstrap concluído.")

# 6) Carregando configurações
config = ud.load_config(PROJECT_ROOT / "config" / "defaults.json",
                        PROJECT_ROOT / "config" / "local.json")
paths = ud.resolve_n1_paths(config, PROJECT_ROOT)
ud.set_random_seed(42)
ud.set_display(200, 120)
logger.info("Config carregada e paths resolvidos.")

[INFO] PROJECT_ROOT: C:\Users\fabio\Projetos DEV\data projects\data-project-template
[INFO] sys.path ok. utils: C:\Users\fabio\Projetos DEV\data projects\data-project-template\utils


SyntaxError: unexpected character after line continuation character (utils_data.py, line 30)

## 🔧 Configuração de Fontes

Nesta etapa são definidos os **arquivos de origem** que servirão de base para o projeto.  
Aqui você informa **quais datasets serão utilizados**, **em qual formato** estão (CSV ou Parquet) e, se houver mais de uma fonte, **como elas se relacionam**.

---

### 🗂️ 1. Exploração das fontes disponíveis

Antes de configurar o dicionário `SOURCES`, é possível listar os arquivos presentes no diretório `data/raw/`:

```python
from utils.utils_data import suggest_source_path, path_of

RAW_DIR = paths.raw_dir
suggest_source_path(RAW_DIR, pattern="*.csv")
```

Esse comando exibe uma tabela com os arquivos encontrados e gera uma **sugestão automática de caminho** para copiar e colar no `SOURCES`.  
Basta arrastar o arquivo para a pasta `data/raw/` e executar o bloco — o nome aparecerá pronto para uso.

---

### ⚙️ 2. Definição das fontes de dados (`SOURCES`)

- Cada entrada do dicionário `SOURCES` representa uma tabela nomeada.  
- A chave (ex.: `"main"`, `"dim_customers"`) identifica a tabela no projeto.  
- Cada item contém:
  - `path`: caminho do arquivo dentro de `data/raw/`.  
  - `format`: formato opcional (`"csv"` ou `"parquet"`). Se omitido, o formato é detectado automaticamente.  
  - `read_opts`: parâmetros de leitura personalizados (`encoding`, `sep`, `low_memory`, etc.).

Exemplo de estrutura:
```python
SOURCES = {
    "main": {
        "path": RAW_DIR / "dataset.csv",
        "read_opts": {"encoding": "utf-8", "sep": ",", "low_memory": False}
    },
    # Exemplo de segunda fonte em CSV:
    # "dim_customers": {
    #     "path": RAW_DIR / "exemplo.csv",
    #     "format": "csv"
    # }
    # Exemplo de fonte em Parquet:
    # "dim_customers": {
    #     "path": RAW_DIR / "customers.parquet",
    #     "format": "parquet"
    # }
}
```

---

### 🔗 3. Configuração opcional de junções (`MERGE_STEPS`)

O DataFrame principal é o definido em `MAIN_SOURCE` (geralmente `"main"`).  
Se existirem outras fontes, você pode configurá-las em `MERGE_STEPS` para realizar merges automáticos:

```python
MAIN_SOURCE = "main"
MERGE_STEPS = [
    # ("dim_customers", "left", "customer_id", "id"),
]
```

Cada item define:
- o nome da tabela secundária (`right_name`),
- o tipo de junção (`how`),
- e as chaves de correspondência (`left_on`, `right_on`).

---

### ✅ 4. Validação e feedback automático

Após definir as fontes, o bloco final realiza uma checagem simples:

- Confirma se o arquivo principal existe.  
- Detecta automaticamente o formato (`CSV` ou `Parquet`).  
- Exibe mensagens de status amigáveis:

```
✅ Fontes configuradas com sucesso.
→ Fonte principal: 'main'  | Arquivo: dataset.csv  | Formato detectado: CSV
→ Nenhum merge configurado (usando apenas a fonte principal).
```

---

> 💡 Resumo
>
>Essa abordagem torna a **configuração de fontes interativa, leve e flexível**:
>- Basta colocar novos arquivos em `data/raw/` e usar `suggest_source_path()` para gerar o caminho.  
>- Não há necessidade de alterar outros blocos do notebook — apenas o `SOURCES` e, se necessário, os merges.  
>- Ideal para projetos exploratórios e notebooks reutilizáveis em diferentes datasets.


In [2]:
import importlib, utils.utils_data as ud

RAW_DIR = paths.raw_dir
ud.suggest_source_path(RAW_DIR, pattern="*.csv")

🔎 Arquivos em C:\Users\fabio\Projetos DEV\data projects\data-project-template\data\raw (filtro: *.csv)


,Arquivo,Extensão,Tamanho (KB),Modificado em
0,dataset.csv,.csv,954.6,2019-09-27 19:30:08


In [3]:
# Obs: só alterar este bloco quando precisar mudar os arquivos ou as chaves de junção.

SOURCES = {
    # nome_da_tabela: {path, (opcional) format: 'csv'|'parquet', (opcional) read_opts}
    "main": {
        "path": RAW_DIR / "dataset.csv",
        # "format": "csv",  # se omitir, detecta pelo sufixo
        "read_opts": {"encoding": "utf-8", "sep": ",", "low_memory": False}
    },
    # Exemplo de segunda fonte em CSV:
    # "dim_customers": {
    #   "path": RAW_DIR / "exemplo.csv",
    #   "format": "csv",  # se omitir, detecta pelo sufixo
    #   "read_opts": {"encoding": "utf-8", "sep": ",", "low_memory": False}
    # }
    
    # Exemplo de fonte em Parquet:
    # "dim_customers": {
    #     "path": RAW_DIR / "customers.parquet",
    #     "format": "parquet"
    # }
}

# Plano de merges (opcional):
# Cada item é um passo: (right_name, how, left_on, right_on)
# O DataFrame base será o SOURCES[MAIN_SOURCE]
MAIN_SOURCE = "main"
MERGE_STEPS = [
    # ("dim_customers", "left", "customer_id", "id"),
]

# Validação
main_path = SOURCES[MAIN_SOURCE]["path"]
main_format = SOURCES[MAIN_SOURCE].get("format") or main_path.suffix.lstrip(".").lower()
print(f"✅ Fontes configuradas com sucesso.")

print(
    f"→ Fonte principal: '{MAIN_SOURCE}'  "
    f"| Arquivo: {main_path.name}  "
    f"| Formato detectado: {main_format.upper()}"
)
if MERGE_STEPS:
    print(f"→ {len(MERGE_STEPS)} etapa(s) de merge configuradas.")
else:
    print("→ Nenhum merge configurado (usando apenas a fonte principal).")

✅ Fontes configuradas com sucesso.
→ Fonte principal: 'main'  | Arquivo: dataset.csv  | Formato detectado: CSV
→ Nenhum merge configurado (usando apenas a fonte principal).


## 📥 Ingestão & Visão Rápida

Nesta etapa, os **datasets definidos em `SOURCES`** são carregados, inspecionados e validados antes de iniciar o tratamento dos dados.  
O objetivo é garantir que as fontes estejam corretamente lidas e que o DataFrame principal (`df`) esteja pronto para seguir no pipeline.

---

### ⚙️ 1. Carregamento das fontes

Cada item de `SOURCES` é percorrido para:
- Ler o arquivo indicado (`path`) conforme o formato (`csv` ou `parquet`);  
- Aplicar as opções de leitura (`read_opts`) definidas no dicionário;  
- Armazenar o DataFrame em `tables[name]`, onde `name` é a chave de identificação da fonte.

A função utilizada é `ud.load_table_simple()`, que automaticamente:
- Detecta o formato pelo sufixo (se não informado);
- Aplica `read_opts` de forma segura;
- Exibe logs amigáveis durante o carregamento.

Após cada leitura, é exibido um **resumo rápido** da fonte carregada, com:
- Número de linhas e colunas;
- Tipos de dados;
- Uso de memória;
- E um relatório de valores nulos (`ud.missing_report()`).

---

### 🔗 2. Definição do DataFrame base e junções opcionais

O dataset principal é definido pela variável `MAIN_SOURCE`.  
Caso existam outras fontes configuradas em `MERGE_STEPS`, o notebook aplica as junções automaticamente via `ud.merge_chain()` — garantindo que cada passo do merge seja logado e validado.

Se nenhuma junção for configurada, o notebook utiliza apenas a fonte principal.

---

### 📊 3. Visão geral final do DataFrame

Após a etapa de ingestão e merges, o notebook gera uma visão consolidada do dataset final (`df`) que seguirá no pipeline.  
São exibidos:
- Estrutura geral (`shape`, colunas, tipos);
- Quantidade de valores nulos por coluna;
- E um log detalhado gravado em `reports/data_preparation.log`.

---

### 🧩 Exemplo de estrutura do processo

```python
tables = {}
for name, cfg in SOURCES.items():
    path = cfg["path"]
    fmt = cfg.get("format")
    read_opts = cfg.get("read_opts", {})
    df_src = ud.load_table_simple(path, fmt, read_opts)
    tables[name] = df_src

    print(f"\n=== {name} ===")
    ov = ud.basic_overview(df_src)
    print(json.dumps(ov, indent=2, ensure_ascii=False))
    display(ud.missing_report(df_src).head(20))

# Base principal e merges
df = tables[MAIN_SOURCE]
if MERGE_STEPS:
    df = ud.merge_chain(df, tables, MERGE_STEPS)
else:
    print(f"[INFO] Usando df base: '{MAIN_SOURCE}' (sem merges).")

# Overview final
overview = ud.basic_overview(df)
logger.info(json.dumps(overview, indent=2, ensure_ascii=False))
display(ud.missing_report(df).head(20))
```

---

> 💡 **Resumo:**  
> - Cada fonte de dados é carregada automaticamente com base na configuração do dicionário `SOURCES`.  
> - A função `ud.load_table_simple()` lida tanto com arquivos CSV quanto Parquet e detecta o formato automaticamente.  
> - O dicionário `tables` mantém todas as fontes acessíveis pelo nome definido (ex.: `"main"`, `"dim_customers"`).  
> - A variável `df` representa o dataset principal que seguirá pelas próximas etapas do pipeline.  
> - Logs e relatórios são salvos para garantir transparência e rastreabilidade do processo.


In [4]:
# 1) Carregar todas as fontes
tables = {}
for name, cfg in SOURCES.items():
    path = cfg["path"]
    fmt = cfg.get("format")              # se None, detecta pelo sufixo
    read_opts = cfg.get("read_opts", {}) # ex.: sep/encoding/low_memory para CSV
    df_src = ud.load_table_simple(path, fmt, read_opts)  # agora com prefixo ud.
    tables[name] = df_src

    # visão rápida por fonte
    print(f"\n=== {name} ===")
    ov = ud.basic_overview(df_src)
    print(json.dumps(ov, indent=2, ensure_ascii=False))
    display(ud.missing_report(df_src).head(20))

# 2) Definir df base e aplicar merges (se configurado)
if MAIN_SOURCE not in tables:
    raise KeyError(f"MAIN_SOURCE '{MAIN_SOURCE}' não encontrado. Fontes disponíveis: {list(tables.keys())}")

df = tables[MAIN_SOURCE]
if MERGE_STEPS:
    df = ud.merge_chain(df, tables, MERGE_STEPS)
    print("\n=== Visão geral (df merged) ===")
else:
    print(f"\n[INFO] Usando df base: '{MAIN_SOURCE}' (sem merges).")

# 3) Overview final do df que segue no pipeline
overview = ud.basic_overview(df)
logger.info(json.dumps(overview, indent=2, ensure_ascii=False))
print(json.dumps(overview, indent=2, ensure_ascii=False))
display(ud.missing_report(df).head(20))


2025-11-01 14:36:50,677 | INFO | [load] path=C:\Users\fabio\Projetos DEV\data projects\data-project-template\data\raw\dataset.csv | format=csv | opts={'encoding': 'utf-8', 'sep': ',', 'low_memory': False}
2025-11-01 14:36:50,678 | INFO | [load_csv] C:\Users\fabio\Projetos DEV\data projects\data-project-template\data\raw\dataset.csv

=== main ===
{
  "shape": [
    7043,
    21
  ],
  "columns": [
    "customerID",
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "tenure",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
    "MonthlyCharges",
    "TotalCharges",
    "Churn"
  ],
  "dtypes": {
    "customerID": "object",
    "gender": "object",
    "SeniorCitizen": "int64",
    "Partner": "object",
    "Dependents": "object",
    "tenure": "int64",
    "PhoneService

,missing_rate,missing_count
customerID,0.0,0
DeviceProtection,0.0,0
TotalCharges,0.0,0
MonthlyCharges,0.0,0
PaymentMethod,0.0,0
PaperlessBilling,0.0,0
Contract,0.0,0
StreamingMovies,0.0,0
StreamingTV,0.0,0
TechSupport,0.0,0



[INFO] Usando df base: 'main' (sem merges).
2025-11-01 14:36:50,759 | INFO | {
  "shape": [
    7043,
    21
  ],
  "columns": [
    "customerID",
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "tenure",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
    "MonthlyCharges",
    "TotalCharges",
    "Churn"
  ],
  "dtypes": {
    "customerID": "object",
    "gender": "object",
    "SeniorCitizen": "int64",
    "Partner": "object",
    "Dependents": "object",
    "tenure": "int64",
    "PhoneService": "object",
    "MultipleLines": "object",
    "InternetService": "object",
    "OnlineSecurity": "object",
    "OnlineBackup": "object",
    "DeviceProtection": "object",
    "TechSupport": "object",
    "StreamingTV": "object",
    "StreamingMovies": "object",
    "C

,missing_rate,missing_count
customerID,0.0,0
DeviceProtection,0.0,0
TotalCharges,0.0,0
MonthlyCharges,0.0,0
PaymentMethod,0.0,0
PaperlessBilling,0.0,0
Contract,0.0,0
StreamingMovies,0.0,0
StreamingTV,0.0,0
TechSupport,0.0,0


## 🔖 Catálogo de DataFrames + df “ativo”

Esta etapa cria um **catálogo centralizado de DataFrames** para gerenciar facilmente todas as tabelas carregadas e derivadas ao longo do projeto.  
O objetivo é permitir que diferentes versões e transformações dos dados sejam armazenadas, nomeadas e acessadas de forma organizada e reprodutível.

---

### ⚙️ 1. Inicialização do catálogo (`TableStore`)

A classe `TableStore` (definida em `utils/utils_data.py`) atua como um **repositório de DataFrames nomeados**.  
Ela é inicializada com o dicionário `tables` (criado na etapa de *Ingestão & Visão Rápida*) e define a tabela principal (`MAIN_SOURCE`) como **tabela ativa**.

Durante a inicialização, o notebook exibe uma mensagem de sucesso semelhante a esta:

```
✅ Catálogo de DataFrames inicializado com sucesso.
→ Total de tabelas carregadas: 1
→ Tabela ativa: 'main'  | Shape: 7032 linhas × 21 colunas
```

E apresenta uma prévia do inventário atual:

| name | rows | cols | memory_mb |
|------|------|-------|-----------|
| main | 7032 | 21    | 1.2       |

---

### 📊 2. Definição do DataFrame ativo (`df`)

A variável `df` recebe a tabela atual através de `T.get()`.  
Esse `df` passa a representar o **DataFrame padrão** que seguirá nas próximas etapas do pipeline (limpeza, tipagem e transformação).

---

### 🧩 3. Gerenciamento de múltiplas tabelas

Novos DataFrames podem ser adicionados ao catálogo a qualquer momento, com nomes descritivos e controle de versão.  
O catálogo também permite alternar entre tabelas e listar todas as disponíveis.

**Principais comandos:**

```python
T.add("churn_raw", df, set_current=True)           # adiciona e define como atual
df = T.use("churn_raw")                            # alterna o df ativo
T.add("features_v1", engenharia_de_atributos(df))  # armazena uma nova derivação
df_features = T["features_v1"]                     # acesso direto por nome
display(T.list())                                  # exibe o inventário completo
```

---

### 🧠 4. Benefício prático

- Mantém o notebook limpo e evita sobrescrever DataFrames importantes.  
- Facilita o reuso e o rastreamento das versões intermediárias dos dados.  
- Ideal para projetos com múltiplas fontes, transformações paralelas ou comparações entre conjuntos tratados.  
- Fornece feedback visual imediato sobre o estado do pipeline, tornando o fluxo mais transparente e interativo.

---

> 💡 **Resumo:**  
> O `TableStore` funciona como uma **memória estruturada de DataFrames**, permitindo adicionar, alternar, versionar e consultar tabelas de forma controlada.  
> A variável `df` representa sempre a **tabela ativa** atual, garantindo consistência nas etapas seguintes do pipeline e visibilidade sobre o progresso do processamento.


In [5]:
from utils.utils_data import TableStore

# 1- Inicializa catálogo com as fontes lidas ('tables') e define a base como atual
T = TableStore(initial=tables, current=MAIN_SOURCE)

# 2 - Conveniência: df = tabela atual (boa prática para seguir no pipeline)
df = T.get()

# 3 - Feedback visual de sucesso
print(f"✅ Catálogo de DataFrames inicializado com sucesso.")
print(f"→ Total de tabelas carregadas: {len(tables)}")
print(f"→ Tabela ativa: '{T.current}'  | Shape: {df.shape[0]} linhas × {df.shape[1]} colunas")

# 4️4 - Exibe uma prévia do catálogo
display(T.list().head())

# --- Guia rápido  ---
# T.add("churn_raw", df, set_current=True)          # cadastra e ativa
# df = T.use("churn_raw")                           # muda o atual e retorna df
# T.add("features_v1", engenharia_de_atributos(df)) # salva uma derivação
# df_features = T["features_v1"]                    # acesso direto por nome
# display(T.list())                                 # inventário das tabelas


✅ Catálogo de DataFrames inicializado com sucesso.
→ Total de tabelas carregadas: 1
→ Tabela ativa: 'main'  | Shape: 7043 linhas × 21 colunas


,name,rows,cols,memory_mb
0,main,7043,21,6.820952


In [6]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## 🧪 Qualidade & Tipagem

Nesta etapa o notebook realiza a **padronização estrutural e tipagem automática dos dados**, garantindo que o DataFrame principal (`df`) siga para as próximas fases **coerente, limpo e otimizado em memória**.  
É uma das fases mais críticas do pipeline, pois evita que inconsistências de tipo e formato prejudiquem as análises posteriores ou a modelagem.

### 📋 O que é feito aqui

1. **Remoção de espaços em branco (`strip_whitespace`)**
   - Elimina espaços extras no início e no fim de valores textuais.  
   - Evita divergências em comparações e agrupamentos (ex.: `"Yes "` ≠ `"Yes"`).  
   - Essa operação é **idempotente**: se os dados já estiverem limpos, não altera nada.

2. **Conversão inteligente de valores numéricos (`infer_numeric_like`)**
   - Detecta colunas com valores *aparentemente numéricos*, mas armazenados como texto (ex.: `"R$ 120,00"` ou `"1.234,56"`).  
   - Aplica heurísticas para remover símbolos, normalizar separadores e converter apenas quando a proporção de conversão for suficiente (`min_ratio = 0.9`).  
   - Evita conversões indevidas (ex.: IDs) por meio de uma lista de exclusão (`blacklist`).  
   - Gera o relatório **`cast_report`**, registrando:
     - Coluna analisada  
     - Ação tomada (convertida, ignorada ou parcial)  
     - Taxa de sucesso e total de valores convertidos.

3. **Otimização de tipos numéricos (`reduce_memory_usage`)**
   - Converte automaticamente `int64` → `int32` e `float64` → `float32`, reduzindo o consumo de memória sem alterar os valores.  
   - O log mostra a diferença antes e depois (ex.: `Memory reduced: 7.76MB → 6.21MB`).

4. **Remoção de duplicatas (`deduplicate_rows`)**
   - Verifica e elimina registros repetidos conforme a configuração (`subset`, `keep`).  
   - Caso encontre duplicatas, gera um relatório detalhado e salva o log em `reports/duplicates.csv`.  
   - Exibe uma amostra das duplicatas detectadas, se houver.

5. **Relatórios e feedback automático**
   - Ao final da execução, exibe um resumo completo:  
     - Linhas e colunas antes/depois  
     - Variação de memória (`Δ MB`)  
     - Relatórios gerados (`cast_report`, `duplicates_report`, etc.)  
   - Todos os logs são gravados em `reports/data_preparation.log`.

---

> 💡 **Resumo:**  
> Esta célula garante que o dataset esteja **consistente, tipado e otimizado**, pronto para análises e modelagem.  
> Corrige formatações, converte números armazenados como texto, remove duplicatas e reduz o uso de memória — tudo com registros automáticos e relatórios salvos para auditoria.


In [7]:
# Executa a etapa de qualidade & tipagem encapsulada no utils
df_before_shape = df.shape
mem_before = float(df.memory_usage(deep=True).sum() / (1024**2))

df, rep = ud.n1_quality_typing(df, config, paths.reports_dir)

mem_after  = float(df.memory_usage(deep=True).sum() / (1024**2))
delta_rows = df.shape[0] - df_before_shape[0]
delta_cols = df.shape[1] - df_before_shape[1]
delta_mem  = mem_after - mem_before

# Exibe relatórios úteis (se existirem)
if "cast_report" in rep:
    print("📄 Conversões numéricas (amostra):")
    display(rep["cast_report"].head(20))

if "duplicates" in rep and isinstance(rep["duplicates"], pd.DataFrame) and not rep["duplicates"].empty:
    print("\n🔁 Duplicatas detectadas (amostra):")
    display(rep["duplicates"].head(10))
    if "duplicates_summary" in rep:
        print("📊 Resumo por chave:")
        display(rep["duplicates_summary"].head(20))
else:
    print("\n✅ Nenhuma duplicidade encontrada segundo os critérios definidos.")

# Mensagem de sucesso consolidada
print(
    f"\n✅ Qualidade & Tipagem concluído com sucesso!\n"
    f"→ Shape: {df_before_shape} → {df.shape} (Δlinhas={delta_rows:+}, Δcolunas={delta_cols:+})\n"
    f"→ Memória: {mem_before:.2f} MB → {mem_after:.2f} MB (Δ={delta_mem:+.2f} MB)\n"
    f"→ Relatórios gerados: {', '.join([k for k,v in rep.items() if isinstance(v, pd.DataFrame) and not v.empty]) or '—'}"
)


2025-11-01 14:36:55,193 | INFO | [N1] strip_whitespace aplicado.
2025-11-01 14:36:55,447 | INFO | [infer_numeric_like] 17 colunas verificadas. Ações: {'skip_no_conversion': 16, 'inplace_convert': 1}
2025-11-01 14:36:55,466 | INFO | [report] salvo em: C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\cast_report.csv
2025-11-01 14:36:55,506 | INFO | Memory reduced: 6.51MB -> 6.36MB
2025-11-01 14:36:55,507 | INFO | [N1] reduce_memory_usage aplicado.
2025-11-01 14:36:55,538 | INFO | [deduplicate] Removed duplicates: 0 (subset=None, keep=first)
2025-11-01 14:36:55,572 | INFO | [reports] salvo em: C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\overview_after_quality.json
📄 Conversões numéricas (amostra):


,column,ratio,action,converted,total,before_dtype,after_dtype
0,Churn,0.0,skip_no_conversion,0,7043,string,string
1,Contract,0.0,skip_no_conversion,0,7043,string,string
2,Dependents,0.0,skip_no_conversion,0,7043,string,string
3,DeviceProtection,0.0,skip_no_conversion,0,7043,string,string
4,InternetService,0.0,skip_no_conversion,0,7043,string,string
5,MultipleLines,0.0,skip_no_conversion,0,7043,string,string
6,OnlineBackup,0.0,skip_no_conversion,0,7043,string,string
7,OnlineSecurity,0.0,skip_no_conversion,0,7043,string,string
8,PaperlessBilling,0.0,skip_no_conversion,0,7043,string,string
9,Partner,0.0,skip_no_conversion,0,7043,string,string



✅ Nenhuma duplicidade encontrada segundo os critérios definidos.

✅ Qualidade & Tipagem concluído com sucesso!
→ Shape: (7043, 21) → (7043, 21) (Δlinhas=+0, Δcolunas=+0)
→ Memória: 6.82 MB → 6.36 MB (Δ=-0.46 MB)
→ Relatórios gerados: cast_report


## 🧼 Padronização Categórica

Nesta etapa ocorre a **padronização global de rótulos textuais** em colunas categóricas, corrigindo inconsistências, diferenças de capitalização e valores redundantes (como `"No internet service"` → `"No"`).  
O objetivo é **garantir uniformidade sem alterar o significado dos dados**, preparando o dataset para codificação e modelagem.

---

### 📋 O que é feito aqui

1. **Seleção automática das colunas categóricas**  
   - São processadas todas as colunas com tipo `object`, `string` ou `category`, exceto aquelas listadas em `exclude` (ex.: `"customerID"`).

2. **Limpeza e normalização de forma**  
   - Remove espaços extras (`trim`) e consolida espaços duplos (`collapse_ws`).  
   - Corrige capitalização conforme o parâmetro `case` (`"title"`, `"lower"`, `"upper"`, `"none"`).  
   - Opcionalmente remove acentos (`strip_accents`).

3. **Substituições e valores nulos**  
   - Aplica um mapa global de substituições (`global_map`) para corrigir padrões conhecidos (ex.: `"No internet service"` → `"No"`).  
   - Permite também um mapa específico por coluna (`per_column_map`), quando necessário.  
   - Converte valores definidos em `null_values` (ex.: `"n/a"`, `"na"`, `"-"`) em `NaN`.

4. **Conversão opcional para categoria**  
   - Se `cast_to_category=True`, converte colunas textuais em `category`, otimizando uso de memória.

5. **Geração de relatório e registro automático**  
   - Cria o arquivo `cat_normalization.csv` dentro da pasta `reports/`, com:  
     - Nome da coluna  
     - Amostras antes/depois  
     - Quantidade de alterações (`changes`)  
     - Número de categorias únicas antes/depois  
   - As informações também são registradas no log e adicionadas ao `manifest.json`.

---

### ⚙️ Configuração (`CAT_NORM_CFG`)

A padronização é controlada por um dicionário de configuração completo:

```python
CAT_NORM_CFG = {
    "enabled": True,
    "exclude": ["customerID"],
    "case": "title",
    "strip_accents": True,
    "collapse_ws": True,
    "trim": True,
    "global_map": {
        "No internet service": "No",
        "No phone service": "No",
        "n/a": "No",
        "none": "No"
    },
    "null_values": ["", "na", "n/a", "-"],
    "cast_to_category": False
}
```

Exemplo de mapa específico por coluna:
```python
"per_column_map": {
    "InternetService": {"Dsl": "DSL", "Fiber Optic": "Fiber Optic"}
}
```

---

> 💡 **Resumo:**  
> Esta célula realiza uma **padronização inteligente e auditável** das colunas categóricas — aplicando limpeza, substituições e normalização de forma sistemática.  
> O processo é controlado via configuração (`CAT_NORM_CFG`), gera um relatório automático e mantém registro completo no log e no `manifest.json`, garantindo **clareza, rastreabilidade e consistência** no tratamento dos dados.


In [8]:
# Config via defaults.json (exemplo seguro e genérico)
CAT_NORM_CFG = {
    "enabled": True,
    "exclude": ["customerID"],         # nunca normalizar IDs
    "case": "title",                   # ou "lower" se preferir
    "strip_accents": True,
    "collapse_ws": True,
    "trim": True,
    "global_map": {
        "No internet service": "No",
        "No phone service": "No",
        "n/a": "No",
        "none": "No"
    },
    "null_values": ["", "na", "n/a", "-"],
    "cast_to_category": False,         # mude pra True se quiser reduzir memória depois
    # "per_column_map": {"PaperlessBilling": {"sim": "Yes", "nao": "No"}}
}

if config.get("normalize_categories", True):
    df, cat_norm_report = ud.normalize_categories(
        df,
        cfg=CAT_NORM_CFG,
        report_path=paths.reports_dir / "cat_normalization.csv"
    )
    if not cat_norm_report.empty:
        print("📑 Relatório de padronização (top 20 por mudanças):")
        display(cat_norm_report.head(20))


2025-11-01 14:36:56,780 | INFO | [report] salvo em: C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\cat_normalization.csv
2025-11-01 14:36:56,802 | INFO | [N1] Padronização categórica aplicada em: ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']
📑 Relatório de padronização (top 20 por mudanças):


,column,before_sample,after_sample,changes,unique_before,unique_after
0,Contract,"Month-to-month, One year, Two year","Month-To-Month, One Year, Two Year",7043,3,3
1,PaymentMethod,"Electronic check, Mailed check, Bank transfer ...","Electronic Check, Mailed Check, Bank Transfer ...",7043,4,4
2,InternetService,"DSL, Fiber optic, No","Dsl, Fiber Optic, No",5517,3,3
3,OnlineSecurity,"No, Yes, No internet service","No, Yes",1526,3,2
4,OnlineBackup,"Yes, No, No internet service","Yes, No",1526,3,2
5,DeviceProtection,"No, Yes, No internet service","No, Yes",1526,3,2
6,TechSupport,"No, Yes, No internet service","No, Yes",1526,3,2
7,StreamingTV,"No, Yes, No internet service","No, Yes",1526,3,2
8,StreamingMovies,"No, Yes, No internet service","No, Yes",1526,3,2
9,MultipleLines,"No phone service, No, Yes","No, Yes",682,3,2


## 🩹 Tratamento de Valores Faltantes

Nesta etapa são tratadas as **ausências de dados** (valores nulos ou `NaN`) para garantir que o DataFrame siga consistente para as próximas fases do pipeline.  
O objetivo é **preencher ou sinalizar valores faltantes** de forma controlada, preservando a integridade estatística das colunas e registrando quais linhas foram modificadas.

### 📋 O que é feito aqui

1. **Geração de relatório inicial**
   - Exibe um diagnóstico das colunas com valores ausentes, mostrando a proporção (`missing_rate`) e o total (`missing_count`) de registros nulos.
   - Essa visualização ajuda a decidir se o tratamento será simples ou se exigirá técnicas específicas.

2. **Aplicação da estratégia de imputação**
   - O comportamento é controlado pela chave `missing_strategy` do arquivo de configuração:
     - `"simple"` → aplica a função `simple_impute_with_flags()`
     - `"advanced"` → reservado para métodos personalizados (ex.: KNN, regressão, interpolação)
   - A estratégia **simples** executa:
     - Substituição de valores nulos **numéricos** pela **mediana** da coluna.  
     - Substituição de valores nulos **categóricos** pela **moda** (valor mais frequente).  
   - Durante a imputação, são criadas **colunas de flag** no formato `was_imputed_<coluna>`, indicando quais linhas foram alteradas.

3. **Relatório final de verificação**
   - Após o preenchimento, é exibido novamente o relatório de faltantes para confirmar se todas as lacunas foram resolvidas.
   - Caso ainda existam colunas críticas, o notebook pode ser ajustado para aplicar métodos mais avançados.

### Exemplo de flag gerada:
| customerID | TotalCharges | was_imputed_TotalCharges |
|-------------|--------------|--------------------------|
| 7590-VHVEG  | 29.85        | False                   |
| 9237-HQITU  | 1840.75      | True                    |

Linhas com `True` indicam que o valor original estava ausente e foi imputado automaticamente.

> 💡 **Resumo:**  
> Esta célula identifica e corrige valores faltantes, aplicando regras de imputação simples e registrando onde cada substituição ocorreu.  
> Isso garante **transparência, rastreabilidade e controle de qualidade**, permitindo filtrar posteriormente apenas os dados considerados consistentes.


In [9]:
if config.get("handle_missing", True):
    print("Relatório de faltantes (antes):")
    display(ud.missing_report(df).head(20))

    if config.get("missing_strategy") == "simple":
        df = ud.simple_impute_with_flags(df)
    else:
        # espaço para técnicas avançadas
        pass

    print("Relatório de faltantes (depois):")
    display(ud.missing_report(df).head(20))

Relatório de faltantes (antes):


,missing_rate,missing_count
TotalCharges,0.001562,11
customerID,0.000000,0
DeviceProtection,0.000000,0
MonthlyCharges,0.000000,0
PaymentMethod,0.000000,0
PaperlessBilling,0.000000,0
Contract,0.000000,0
StreamingMovies,0.000000,0
StreamingTV,0.000000,0
TechSupport,0.000000,0


2025-11-01 14:36:58,920 | INFO | [impute] 'TotalCharges' → 11 valores (mediana).
Relatório de faltantes (depois):


,missing_rate,missing_count
customerID,0.0,0
gender,0.0,0
Churn,0.0,0
TotalCharges,0.0,0
MonthlyCharges,0.0,0
PaymentMethod,0.0,0
PaperlessBilling,0.0,0
Contract,0.0,0
StreamingMovies,0.0,0
StreamingTV,0.0,0


## 🚩 Detecção de Outliers

Esta etapa identifica **valores atípicos** nas colunas numéricas, adicionando colunas de flag (`*_is_outlier`) que indicam quais registros se desviam do padrão estatístico esperado.  
Essas colunas servem para **inspeção e auditoria**, sem alterar ou remover dados — a decisão de tratamento posterior (ajuste, exclusão ou manutenção) é **de negócio**.

### ⚙️ Como funciona

1. **Verifica a configuração**
   - A execução depende da flag `detect_outliers` no arquivo `config/defaults.json`.  
     Se estiver definida como `true`, a detecção é realizada.

2. **Seleciona o método estatístico**
   - `"iqr"` → usa o **Intervalo Interquartil (Interquartile Range)**:
     - Calcula Q1 (25º percentil) e Q3 (75º percentil)
     - Define limites:  
       Inferior = Q1 − 1.5 × IQR  
       Superior = Q3 + 1.5 × IQR
     - Valores fora desse intervalo são marcados como outliers.
   - `"zscore"` → usa o **desvio-padrão (Z-Score)**:
     - Calcula a média (μ) e o desvio-padrão (σ)
     - Para cada valor, obtém `z = (x − μ) / σ`
     - Valores com |z| > 3 são considerados outliers.

3. **Criação de colunas de flag**
   - Para cada variável numérica analisada, é criada uma nova coluna:
     ```
     <coluna>_is_outlier
     ```
   - O valor será:
     - `True` → registro identificado como outlier  
     - `False` → registro dentro do intervalo esperado

4. **Registro no log**
   - Ao final, o sistema registra no log quantas colunas de flag foram criadas:
     ```
     Outlier flags created: 4
     ```
     Significa que 4 colunas numéricas foram analisadas e receberam suas respectivas flags.

### Exemplo de resultado

| tenure | MonthlyCharges | TotalCharges | tenure_is_outlier | MonthlyCharges_is_outlier | TotalCharges_is_outlier |
|--------|----------------|--------------|-------------------|----------------------------|--------------------------|
| 5      | 35.50          | 190.00       | False             | False                      | False                    |
| 72     | 120.00         | 9999.99      | False             | **True**                   | **True**                 |

> 💡 **Resumo:**  
> Esta célula adiciona colunas de marcação para detectar valores atípicos de acordo com o método configurado.  
> Os dados originais são preservados, permitindo que a análise posterior defina se esses registros devem ser **mantidos, ajustados ou removidos**.


In [14]:
df, out_info = ud.apply_outlier_flags(df, config=config, persist=True)
logger.info(f"Outlier flags created: {out_info['created_flags']}")
# visão rápida
if out_info['created_flags'] > 0:
    display(
        df.filter(like="_is_outlier")
          .sum()
          .sort_values(ascending=False)
          .to_frame("outliers_count")
          .head(20)
    )


2025-11-01 14:50:28,253 | INFO | [outliers] IQR aplicado (4 colunas).
2025-11-01 14:50:28,270 | INFO | Outlier flags created: 4


In [15]:
display(df.filter(like="_is_outlier").sum())

SeniorCitizen_is_outlier     1142
tenure_is_outlier               0
MonthlyCharges_is_outlier       0
TotalCharges_is_outlier         0
dtype: int64

## 🧬 Duplicidades

Remove **linhas duplicadas** do DataFrame para evitar contagens infladas, vieses e ruído em métricas.

### 📋 O que acontece aqui
- Por padrão, a duplicidade é verificada **linha a linha** (todas as colunas iguais).  
- A primeira ocorrência é mantida e as demais são removidas.

### 🔧 Opções (se configuradas no `config`)
- `deduplicate_subset`: lista de colunas que definem a chave de deduplicação  
  *Ex.:* `["customerID"]` mantém apenas um registro por cliente.
- `deduplicate_keep`: política de retenção — `"first"`, `"last"` ou `false` (remove todas as repetições).
- `deduplicate_log` + `deduplicate_log_filename`: salva um **CSV** com as duplicatas detectadas em `reports/`.

### Boas práticas:
- Uso do `subset` quando a duplicidade for **conceitual** (ex.: mesma pessoa/pedido), não necessariamente toda a linha idêntica.
- Gerar e revisar o relatório de duplicatas antes de decidir pela remoção definitiva.

> 💡**Resumo:** esta etapa garante um dataset **sem registros repetidos** segundo o critério definido, mantendo rastreabilidade quando o log estiver habilitado.

In [16]:
if config['deduplicate']:
    df = deduplicate_rows(df)

2025-10-30 06:01:08,643 | INFO | deduplicate_rows: subset=None → desduplicando linha inteira
2025-10-30 06:01:08,659 | INFO | deduplicate_rows: removidas 0 duplicatas (de 7043)


## 🛠️ Engenharia de Atributos

Criação de novas colunas que capturem **relações, proporções, categorias ou padrões** relevantes ao negócio.  
Essa etapa é inteiramente manual e depende do contexto do dataset.

### 💡 Exemplos:
- **Razões e proporções:** `TotalCharges / tenure` → gasto médio mensal.  
- **Flags categóricas:** `Contract == 'Month-to-month'` → 1 se contrato mensal.  
- **Contagens de serviços:** soma de colunas binárias (Yes/No).  

Essas novas features tornam as análises mais ricas e **melhoram a performance de modelos preditivos**.


In [17]:
if config['feature_engineering']:
    # Exemplo genérico (comente/remova conforme o caso):
    # if {'col_a','col_b'}.issubset(df.columns):
    #     df['a_per_b'] = df['col_a'] / df['col_b'].replace(0, np.nan)
    pass

# 📅 Tratamento de Datas

Esta etapa detecta e converte colunas de datas de forma **automática e controlada**, criando **features temporais** úteis para análises e modelos preditivos.

---

## 📋 O que acontece aqui

### 🔹 Identificação de colunas de data

O sistema procura automaticamente colunas com nomes que contenham termos como  
`date`, `data`, `dt_`, `_dt`, ou `_date`.

Também é possível **forçar colunas específicas** definindo manualmente em:

```python
date_cfg["explicit_cols"] = ["StartDate", "EndDate"]
```

### 🔹 Conversão para datetime com auditoria

Cada coluna candidata é testada com diferentes formatos e tentativas de *parsing*.  
O relatório **`parse_report`** mostra:

| column      | parsed_ratio | converted |
|--------------|--------------|------------|
| order_date   | 1.00         | True       |
| start_date   | 0.35         | False      |

- **column:** nome da coluna testada  
- **parsed_ratio:** porcentagem de valores convertidos com sucesso  
- **converted:** indica se a coluna foi convertida (baseado no `min_ratio`)

---

### 🔹 Criação automática de features temporais

Para cada coluna convertida em `datetime`, são geradas variáveis derivadas como:

`*_year`, `*_month`, `*_day`, `*_dayofweek`, `*_quarter`, `*_week`,  
`*_is_month_start`, `*_is_month_end`

O prefixo das novas colunas é o **nome original da coluna de data** .

---

### 🧱 Comportamento defensivo

Caso nenhuma coluna seja detectada, a célula não gera erro — apenas loga:

`[dates] Nenhuma coluna de data detectada/convertida. Pule a criação de features.`


Assim, o pipeline segue normalmente.

---

### ⚙️ Configuração (`date_cfg`)

| Parâmetro       | Descrição                                                        | Exemplo               |
|-----------------|------------------------------------------------------------------|------------------------|
| `detect_regex`  | Padrão para localizar colunas com nomes de data                 | `"date"`              |
| `explicit_cols` | Lista manual de colunas a converter                             | `["StartDate"]`       |
| `dayfirst`      | Define se o formato é D/M/Y                                     | `True` para 🇧🇷        |
| `utc`           | Define se a conversão deve ser em UTC                           | `False`               |
| `formats`       | Lista de formatos específicos                                   | `["%d/%m/%Y"]`        |
| `min_ratio`     | Fração mínima de parsing bem-sucedido para aceitar a conversão  | `0.8`                 |

---

> 💡 **Resumo:**  
> Esta célula realiza o reconhecimento automático de colunas de data,
converte-as para o formato datetime e gera variáveis derivadas como
ano, mês, dia e semana.
> Caso nenhuma coluna de data seja encontrada, o código é ignorado com segurança,
garantindo a continuidade do pipeline sem erros.
> Os parâmetros em date_cfg permitem ajustar formato, localização e tolerância
de parsing conforme a estrutura de cada dataset.

In [18]:
date_cfg = {
  "detect_regex": r"(date|data|dt_|_dt$|_date$)",
  "explicit_cols": [],   # ex.: ["StartDate","EndDate"]
  "dayfirst": False,     # True se datas forem D/M/Y
  "utc": False,
  "formats": [],         # ex.: ["%d/%m/%Y", "%Y-%m-%d"]
  "min_ratio": 0.80,
}


# Converter e auditar parsing
df, parse_report, parsed_cols = parse_dates_with_report(df, date_cfg)
display(parse_report)

if not parsed_cols:
    logger.info("[dates] Nenhuma coluna de data detectada/convertida. Pule a criação de features.")
else:
    created = expand_date_features(
        df, parsed_cols,
        features=["year","month","day","dayofweek","quarter","week","is_month_start","is_month_end"],
        prefix_mode="auto"
    )
    logger.info(f"[dates] features criadas: {len(created)}")

2025-10-30 06:01:19,351 | INFO | [dates] candidates=[]
2025-10-30 06:01:19,352 | INFO | [dates] parsed_ok=[]


,column,parsed_ratio,converted


2025-10-30 06:01:19,356 | INFO | [dates] Nenhuma coluna de data detectada/convertida. Pule a criação de features.


# 🗓️ Criação da Tabela Calendário (`dim_date`)

Esta etapa gera automaticamente uma **tabela calendário completa** — também chamada de **dimensão de tempo** — a partir de uma coluna de datas existente no dataset principal.  
A tabela é útil para **análises temporais, dashboards e modelos de previsão** que utilizam períodos como referência (ano, mês, trimestre, etc).

---

## 📋 O que acontece aqui

### 🔹 Seleção da coluna de data

Você define manualmente qual coluna será usada como base:

```python
CAL_DATE_COL = "order_date"  # nome da coluna datetime escolhida
CAL_FREQ = "D"               # frequência: "D" (diário), "W" (semanal), "M" (mensal)
```

A coluna deve estar no formato **datetime**.  
Se não estiver, será exibido um erro pedindo para executar antes a etapa de **Tratamento de Datas**.

---

### 🔹 Geração da dimensão calendário

A função `build_calendar_from()` constrói uma tabela com todas as datas entre o **mínimo** e o **máximo** encontrados em `CAL_DATE_COL`.

Para cada data, são criadas colunas derivadas, como:

| Coluna          | Descrição                                |
|------------------|-------------------------------------------|
| `date`           | Data base (chave principal)              |
| `year`           | Ano                                      |
| `month`          | Mês numérico                             |
| `day`            | Dia do mês                               |
| `quarter`        | Trimestre (1–4)                          |
| `week`           | Semana do ano (ISO)                      |
| `dow`            | Dia da semana (0 = segunda, 6 = domingo) |
| `is_month_start` | Indica se a data é o primeiro dia do mês |
| `is_month_end`   | Indica se a data é o último dia do mês   |
| `month_name`     | Nome do mês                              |
| `day_name`       | Nome do dia da semana                    |

---

### 🔹 Armazenamento e reuso

A tabela é salva automaticamente no diretório de artefatos (`artifacts/`):

```python
CAL_OUT = ARTIFACTS_DIR / "dim_date.csv"
```

O formato é determinado pela extensão (.csv ou .parquet).
Além disso, a tabela pode ser registrada no catálogo de DataFrames (T) para uso posterior no pipeline:
```python
T.add("dim_date", dim_date)
```
> 💡 **Resumo:**  
> Esta célula cria uma dimensão de tempo completa baseada na coluna de data escolhida.
> Ela facilita comparações e agregações por ano, mês, semana ou trimestre, além de permitir junções temporais consistentes com outros datasets ou dashboards (ex.: Power BI).
> O processo é automático, reproduzível e independente do dataset principal.

In [19]:
# Ajuste estes parâmetros conforme o dataset:
CAL_DATE_COL = "order_date"           # escolha aqui a coluna de datas já convertida para datetime
CAL_FREQ     = "D"                    # "D" (diário), "W" (semanal), "M" (mensal) etc.
CAL_OUT      = ARTIFACTS_DIR / "dim_date.csv"  # caminho de saída (csv/parquet conforme a extensão)

# --- validações suaves ---
if CAL_DATE_COL not in df.columns:
    msg = f"⚠️ Coluna '{CAL_DATE_COL}' não encontrada no DataFrame. " \
          f"Ajuste CAL_DATE_COL para uma coluna válida antes de gerar a tabela calendário."
    print(msg)
    logger.warning(msg)
else:
    if not pd.api.types.is_datetime64_any_dtype(df[CAL_DATE_COL]):
        msg = f"⚠️ Coluna '{CAL_DATE_COL}' existe, mas **não está em formato datetime**. " \
              "Execute a etapa de Tratamento de Datas antes, ou converta manualmente."
        print(msg)
        logger.warning(msg)
    else:
        # --- construção da dimensão calendário ---
        dim_date = build_calendar_from(df, CAL_DATE_COL, freq=CAL_FREQ)

        # --- visão rápida ---
        display(dim_date.head(12))
        start_date = dim_date["date"].min()
        end_date   = dim_date["date"].max()
        print(f"Período: {start_date.date()} → {end_date.date()}  | Linhas: {len(dim_date)}")

        # --- salvar em disco respeitando a extensão do caminho ---
        save_table(dim_date, CAL_OUT)
        logger.info(f"[calendar] Tabela calendário salva em: {CAL_OUT}")

        # --- opcional: registrar no catálogo de tabelas ---
        try:
            T.add("dim_date", dim_date)
            logger.info("[calendar] 'dim_date' registrada no catálogo T.")
        except NameError:
            # Se TableStore (T) não estiver sendo usado nesta sessão, ignore.
            pass


⚠️ Coluna 'order_date' não encontrada no DataFrame. Ajuste CAL_DATE_COL para uma coluna válida antes de gerar a tabela calendário.
2025-10-30 06:01:24,339 | WARNING | ⚠️ Coluna 'order_date' não encontrada no DataFrame. Ajuste CAL_DATE_COL para uma coluna válida antes de gerar a tabela calendário.


# 📝 Tratamento de Texto (opcional)

Esta etapa extrai **métricas numéricas e lógicas a partir de colunas textuais**, transformando texto livre em informações quantitativas úteis para **análise exploratória e modelagem**.  

É uma forma leve e controlada de **estruturar dados não numéricos**, sem recorrer a técnicas avançadas de NLP.

---

## 📋 O que acontece aqui

### 🔹 Identificação de colunas textuais

O sistema busca automaticamente colunas com tipo `object` e ignora aquelas listadas na *blacklist*:

```python
text_cols = [c for c in df.columns if df[c].dtype == 'object' and c not in TEXT_CFG["blacklist"]]
```

Essas colunas normalmente contêm informações como:  
descrições, comentários, nomes, categorias textuais ou observações.

---

### 🔹 Limpeza e padronização

Antes de gerar as métricas, os textos passam por uma limpeza leve:
- Remoção de **espaços duplicados** e **trim** nas extremidades.  
- Conversão para **minúsculas**, garantindo consistência em análises de termos.

Essas ações são controladas pelas chaves:
```python
"lower": True,
"strip_collapse_ws": True
```

---

### 🔹 Criação automática de métricas textuais

Para cada coluna textual, são geradas novas colunas numéricas:

| Nova Coluna           | Descrição | Exemplo ("This is great!") |
|------------------------|------------|-----------------------------|
| `<coluna>_len`         | Número total de caracteres no texto | 15 |
| `<coluna>_word_count`  | Número de palavras separadas por espaços | 3 |
| `<coluna>_alpha_count` | Número de letras (A–Z, a–z) | 13 |
| `<coluna>_digit_count` | Número de dígitos numéricos | 0 |

---

### 🔹 Presença de termos-chave

O sistema também cria colunas booleanas (`True` / `False`) para identificar a **ocorrência de palavras específicas** em cada coluna textual.  

Exemplo:
```python
TEXT_CFG["keywords"] = ["error", "cancel", "premium"]
```

Gera colunas como:
- `<coluna>_has_error`  
- `<coluna>_has_cancel`  
- `<coluna>_has_premium`

Essas variáveis são úteis para análises de sentimento ou padrões de ocorrência em feedbacks de clientes.

---

### 🔹 Exportação de resumo

Ao final, é gerado um **resumo CSV** com as colunas de texto processadas e suas features derivadas.  
O arquivo é salvo em:
```
reports/text_features/summary.csv
```

Esse relatório documenta as transformações aplicadas, garantindo **rastreabilidade e transparência**.

---

### ⚙️ Configuração (`TEXT_CFG`)

| Parâmetro              | Descrição                                                       | Exemplo                              |
|------------------------|------------------------------------------------------------------|--------------------------------------|
| `lower`                | Converte o texto para minúsculas                                | `True`                               |
| `strip_collapse_ws`    | Remove espaços duplicados e limpa extremidades                  | `True`                               |
| `keywords`             | Lista de termos a detectar no texto                             | `["error", "cancel", "premium"]`     |
| `blacklist`            | Colunas a ignorar durante o processamento                       | `["customerID"]`                     |
| `export_summary`       | Salva relatório com resumo das features geradas                 | `True`                               |

---

### Boas práticas

- Aplicar esta etapa apenas em colunas realmente textuais — evite IDs ou códigos.  
- Personalizar a lista de **palavras-chave** conforme o contexto do dataset.  
- Caso o dataset não possua colunas de texto, o processo será ignorado com segurança.  
- Utilize o arquivo de resumo (`summary.csv`) para acompanhar colunas derivadas e auditorias.

---

> 💡 **Resumo:**  
> Esta célula transforma campos de texto em **indicadores numéricos e lógicos**, gerando métricas básicas (tamanho, palavras, letras, dígitos) e flags de presença de termos-chave.  
> Tudo é processado automaticamente com **configuração leve e reprodutível**, integrando dados textuais ao pipeline de forma organizada e escalável.


In [11]:
TEXT_CFG = {
    "lower": True,
    "strip_collapse_ws": True,
    "keywords": ["error", "cancel", "premium"],
    "blacklist": ["customerID"],
    "export_summary": True,
}

if config.get("text_features", True):
    df, text_summary = extract_text_features(
        df,
        lower=TEXT_CFG["lower"],
        strip_collapse_ws=TEXT_CFG["strip_collapse_ws"],
        keywords=TEXT_CFG["keywords"],
        blacklist=TEXT_CFG["blacklist"],
        export_summary=TEXT_CFG["export_summary"],
        summary_dir=REPORTS_DIR / "text_features"
    )
    display(text_summary.head())


2025-10-30 08:36:56,991 | INFO | [text] colunas processadas: ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn']
2025-10-30 08:36:56,992 | INFO | [text] features criadas: 112
2025-10-30 08:36:57,006 | INFO | [text] resumo salvo em: C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\text_features\summary.csv
📝 Resumo das colunas textuais (amostra):


,text_col,keywords_cols
0,gender,"gender_has_error, gender_has_cancel, gender_ha..."
1,Partner,"Partner_has_error, Partner_has_cancel, Partner..."
2,Dependents,"Dependents_has_error, Dependents_has_cancel, D..."
3,PhoneService,"PhoneService_has_error, PhoneService_has_cance..."
4,MultipleLines,"MultipleLines_has_error, MultipleLines_has_can..."


# 🔤 Codificação de Categóricas & 🔢 Escalonamento Numérico (opcionais)

Esta etapa transforma variáveis **categóricas** em representações numéricas e **padroniza a escala** das variáveis **numéricas** quando necessário.  
É útil para alimentar modelos de ML que **não aceitam strings** (ex.: regressões, SVMs, redes neurais) e/ou **são sensíveis à escala** (KNN, SVM com kernel RBF, PCA, etc.).

---

### 🧭 Visão Geral

- **Codificação (Encoding)**
  - **One-hot**: cria uma coluna por categoria (0/1). É a opção **mais segura** para baseline; evita ordenar categorias artificialmente.
  - **Ordinal**: mapeia categorias para números inteiros. Mais **compacto**, mas induz **ordem artificial**; use com cuidado.

- **Escalonamento (Scaling)**
  - **Standard**: `z = (x - média) / desvio`; centra em 0 com variância ≈ 1. Bom para dados **aprox. gaussianos**.
  - **MinMax**: escala para **[0, 1]**; útil quando limites mínimos e máximos são relevantes (redes neurais, normalizações simples).

---

### ⚙️ Configuração usada no notebook

Estas configurações são lidas do `config/defaults.json` (com `local.json` sobrepondo, se existir) e são repassadas como dicionários para a função `apply_encoding_and_scaling`:

```python
ENCODE_CFG = {
    "enabled":           config.get("encode_categoricals", True),
    "type":              config.get("encoding_type", "onehot"),  # "onehot" | "ordinal"
    "exclude_cols":      ["Churn", "customerID"],                # NÃO codificar alvo/ids
    "high_card_threshold": 50,                                    # ignora colunas com cardinalidade muito alta
}

SCALE_CFG = {
    "enabled":           config.get("scale_numeric", False),
    "method":            config.get("scaler", "standard"),       # "standard" | "minmax"
    "exclude_cols":      ["Churn"],                              # NÃO escalar alvo
    "only_continuous":   True,                                   # evita escalar dummies e inteiros-discretos
}

df, encoding_meta, scaling_meta = apply_encoding_and_scaling(
    df, encode_cfg=ENCODE_CFG, scale_cfg=SCALE_CFG
)
```

> 💡 **Dica:** deixe `scale_numeric = false` no início do projeto (exploração/entendimento). Ative a escala somente quando partir para a **modelagem** e **validação**.

---

### 🧩 O que a função faz (alto nível)

1. **Seleciona colunas categóricas** (`object`/`category`) **excluindo** as listadas em `exclude_cols` e as com **cardinalidade > high_card_threshold** (proteção contra explosão de dummies).
2. **Codifica** conforme `type`:
   - `onehot` → gera `get_dummies` com `dtype=float` e concatena ao DataFrame (removendo as originais).
   - `ordinal` → aplica `OrdinalEncoder` do scikit-learn (com `unknown_value=-1`).
3. **Escala colunas numéricas** se `SCALE_CFG["enabled"]`:
   - seleciona **apenas** colunas numéricas **contínuas** quando `only_continuous=True` (float/large-range).
   - aplica `StandardScaler` **ou** `MinMaxScaler`.
4. Retorna:
   - `df` (atualizado),
   - `encoding_meta` (categorias vistas, tipo de codificação, colunas excluídas, descartes por cardinalidade),
   - `scaling_meta` (tipo de escala, colunas escaladas, parâmetros do scaler).

---

### 📦 Saídas e Metadados

- **`encoding_meta`** inclui:
  - `encoding`: `"onehot"` **ou** `"ordinal"`
  - `excluded`: lista de colunas ignoradas (ex.: `["Churn","customerID"]`)
  - `high_card_excluded`: colunas **não encodadas** por alta cardinalidade
  - `categorical_columns`: colunas categóricas processadas
  - (para ordinal) `categories_`: categorias aprendidas por coluna

- **`scaling_meta`** inclui:
  - `scaler`: `"standard"` **ou** `"minmax"`
  - `scaled_columns`: lista das colunas escaladas
  - `means_`/`scales_` (para Standard) **ou** `min_`/`range_` (para MinMax) — úteis para **reprodução** no deploy

Esses metadados são importantes para **reaplicar** transformações de forma consistente em produção (ou na inferência).

---

### 🧪 Exemplo de comportamento esperado

Suponha um dataset com colunas:
- `gender` (`Male`/`Female`), `InternetService` (3 categorias), `MonthlyCharges` (float), `tenure` (int), `Churn` (alvo).

Com `onehot` + `standard` e exclusões padrão, o resultado será:
- Novas colunas como `gender_Female`, `gender_Male`, `InternetService_DSL`, etc.
- `MonthlyCharges` escalado (média≈0, desvio≈1), `tenure` **pode** ser ignorado se `only_continuous=True` e for considerado discreto.

---

### 🚧 Armadilhas comuns e proteções do template

- **Explosão de dummies**: colunas com **muitas categorias** são ignoradas (registro em `high_card_excluded`).
- **Vazar o alvo**: `exclude_cols` deve conter a **target** (ex.: `"Churn"`).
- **IDs**: evite codificar/escale IDs (`customerID`); eles não carregam semântica útil.
- **Mixed types**: colunas mal tipadas (número como string) devem ser tratadas antes (use a célula **Qualidade & Tipagem**).

---

### ✅ Status de execução e logs

Ao executar, você verá mensagens no log do tipo:

```
[encode] type=onehot | cols=['gender', 'InternetService'] | high_card_excluded=[]
[scale] method=standard | scaled_cols=['MonthlyCharges']
```

Se o log mostrar `cols=[]`, significa que **não há colunas categóricas** elegíveis (ou foram excluídas por configuração/cardinalidade). Isso é **normal** em alguns datasets.

---

### 🧠 Boas práticas

- Comecar com **one-hot** e **sem escala** para ter uma baseline interpretável.
- Ativar **ordinal** apenas quando existir **ordem natural** nas categorias.
- Escalonar **depois** de separar **treino/validação** para evitar vazamento (no template, esta etapa é opcional e controlada por flag).
- Guardar `encoding_meta` e `scaling_meta` se for levar o modelo para produção.

---

> 💡 **Resumo:**  
> Esta célula converte dados categóricos e numéricos em formatos ideais para modelagem, mantendo controle sobre exclusões, cardinalidade e escala — garantindo robustez, consistência e clareza no tratamento dos dados.

In [24]:
import importlib, utils.utils_data as ud
importlib.reload(ud)  # garante a versão mais recente

TEXT_CFG = {
    "lower": True,
    "strip_collapse_ws": True,
    "keywords": ["error", "cancel", "premium"],
    "blacklist": ["customerID"],
    "export_summary": True,
}

if config.get("text_features", True):
    df, text_summary = ud.extract_text_features(
        df,
        lower=TEXT_CFG["lower"],
        strip_collapse_ws=TEXT_CFG["strip_collapse_ws"],
        keywords=TEXT_CFG["keywords"],
        blacklist=TEXT_CFG["blacklist"],
        export_summary=TEXT_CFG["export_summary"],
        summary_dir=REPORTS_DIR / "text_features"
    )
    display(text_summary.head(10))


2025-10-30 06:05:32,736 | INFO | [text] colunas processadas: []
2025-10-30 06:05:32,737 | INFO | [text] features criadas: 0
2025-10-30 06:05:32,744 | INFO | [text] resumo salvo em: C:\Users\fabio\Projetos DEV\data projects\data-project-template\reports\text_features\summary.csv


""


In [25]:
#CRIAÇÃO DE COLUNA TARGET

In [26]:
# Cria o target a partir da config (ou defaults)
df, target_name, class_map, tgt_report = ud.ensure_target_from_config(df, config, verbose=True)

# Opcional: ver um mini-relatório
display(tgt_report)

# Garantir que o N2 enxergue o mapeamento, se você quiser usar class_map lá:
globals()["class_map"] = class_map


2025-10-30 06:05:34,838 | INFO | [target] 'target' criado de 'Churn' → pos=1869 neg=5174 nulls=0 total=7043
[target] Definido target_column='target' (fonte='Churn')


,source_col,target_col,dtype,positives,negatives,nulls,total
0,Churn,target,int,1869,5174,0,7043


## 💾 Exportação de Artefatos

Salve dados **intermediários** e **prontos** para uso em modelagem/visualização.  
Também salvamos um **manifest** com metadados das transformações.


In [27]:
# =============================================================================
# 📦 Exportação de Artefatos (N1 → N2)
# =============================================================================
from datetime import datetime
from pathlib import Path
import json
import numpy as np
import pandas as pd

# 1) Caminhos a partir da config (com defaults)
OUTPUT_PROCESSED = Path(config.get("data_processed_file", "data/processed/processed.parquet"))
OUTPUT_INTERIM   = Path(config.get("data_interim_file",   "data/interim/interim.parquet"))
META_FILE        = Path(config.get("meta_file",           "artifacts/metadata/dataset_meta.json"))

# 2) Garante diretórios
OUTPUT_PROCESSED.parent.mkdir(parents=True, exist_ok=True)
OUTPUT_INTERIM.parent.mkdir(parents=True, exist_ok=True)
META_FILE.parent.mkdir(parents=True, exist_ok=True)
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)  # deve existir na sua célula anterior

# 3) Salva tabelas respeitando a extensão (.csv / .parquet / .xlsx)
def _save_df(df_, path_: Path):
    ext = path_.suffix.lower()
    if ext == ".parquet":
        df_.to_parquet(path_, index=False)
    elif ext == ".csv":
        df_.to_csv(path_, index=False, encoding="utf-8")
    elif ext == ".xlsx":
        df_.to_excel(path_, index=False)
    else:
        raise ValueError(f"Extensão não suportada: {ext}")

if config.get("export_interim", True):
    _save_df(df, OUTPUT_INTERIM)

if config.get("export_processed", True):
    _save_df(df, OUTPUT_PROCESSED)

# 4) Monta METADADOS para o N2
#    - target: vem da config (ou você pode setar manualmente)
target_col = config.get("target_column", "target")

# Listas de colunas
all_cols = df.columns.tolist()
ignored_cols = config.get("ignored_columns", [])  # opcional na sua config
candidate_features = [c for c in all_cols if c not in ignored_cols and c != target_col]

numeric_cols = df[candidate_features].select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = [c for c in candidate_features if c not in numeric_cols]
boolean_cols = df[candidate_features].select_dtypes(include=["bool", "boolean"]).columns.tolist()

# (Se quiser registrar o mapeamento de classes do target, defina aqui; opcional)
class_map = None
# Exemplo (apenas se fizer sentido no seu dataset):
# classes = sorted(df[target_col].dropna().unique().tolist())
# class_map = {c: i for i, c in enumerate(classes)}
# df[target_col] = df[target_col].map(class_map)

meta = {
    "dataset_name": config.get("dataset_name", "Dataset"),
    "version": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "target": target_col,
    "class_map": class_map,
    "columns": {
        "numeric": numeric_cols,
        "categorical": categorical_cols,
        "boolean": boolean_cols,
        "ignored": ignored_cols,
        "all": all_cols
    }
}

# 5) Salva META
META_FILE.write_text(json.dumps(meta, indent=2, ensure_ascii=False), encoding="utf-8")

# 6) Manifest opcional (resumo da execução do N1)
manifest = {
    "created_at": datetime.now().isoformat(timespec="seconds"),
    "random_seed": RANDOM_SEED,
    "config": config,
    "memory_mb": float(df.memory_usage(deep=True).sum() / (1024**2)),
    "outlier_flags": [c for c in df.columns if c.endswith("_is_outlier")],
    "imputed_flags": [c for c in df.columns if c.startswith("was_imputed_")],
    "shape": tuple(df.shape),
    "exported": {
        "interim": str(OUTPUT_INTERIM) if config.get("export_interim", True) else None,
        "processed": str(OUTPUT_PROCESSED) if config.get("export_processed", True) else None,
        "meta_file": str(META_FILE)
    }
}
(ARTIFACTS_DIR / "manifest.json").write_text(json.dumps(manifest, indent=2, ensure_ascii=False), encoding="utf-8")

print("Arquivos gerados:")
print(f"- INTERIM:   {OUTPUT_INTERIM if config.get('export_interim', True) else '(pulado)'}")
print(f"- PROCESSED: {OUTPUT_PROCESSED if config.get('export_processed', True) else '(pulado)'}")
print(f"- META:      {META_FILE}")


Arquivos gerados:
- INTERIM:   data\interim\interim.parquet
- PROCESSED: data\processed\processed.parquet
- META:      artifacts\metadata\dataset_meta.json


## ✅ Checkpoint

- **Revisar** as colunas derivadas e decisões (imputação, outliers, codificação).  
- **Documentar** no README as escolhas de negócio e justificativas.  

## 📎 Anotações

Esta seção pode ser usada como bloco livre para observações específicas do projeto.
